In [1]:
from laboneq.simple import *
from helpers.example_notebook_helper import *
from helpers.example_notebook_simple import create_device_setup
#from zhinst.toolkit import Session
from config import config
name = "q0"

In [2]:
def make_device_setup():
    descriptor=f"""\
    instrument_list:
        SHFQA:
        - address: DEV12096
          uid: device_shfqa
        SHFSG:
        - address: DEV12173
          uid: device_shfsg
        PQSC:  
        - address: DEV10084
          uid: device_pqsc

    connections:
        device_shfqa:    
            - iq_signal: q0/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q0/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q1/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q1/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q2/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q2/acquire_line
              ports: [QACHANNELS/0/INPUT] 
            - iq_signal: q3/measure_line
              ports: [QACHANNELS/0/OUTPUT]
            - acquire_signal: q3/acquire_line
              ports: [QACHANNELS/0/INPUT]

        device_shfsg:
            - iq_signal: q0/drive_line
              ports: SGCHANNELS/0/OUTPUT
            - iq_signal: q0/drive_line_ef
              ports: SGCHANNELS/0/OUTPUT
            - iq_signal: q1/drive_line
              ports: SGCHANNELS/1/OUTPUT
            - iq_signal: q1/drive_line_ef
              ports: SGCHANNELS/1/OUTPUT
            - iq_signal: q2/drive_line
              ports: SGCHANNELS/2/OUTPUT
            - iq_signal: q2/drive_line_ef
              ports: SGCHANNELS/2/OUTPUT
            - iq_signal: q3/drive_line
              ports: SGCHANNELS/3/OUTPUT
            - iq_signal: q3/drive_line_ef
              ports: SGCHANNELS/3/OUTPUT
            - iq_signal: q4/drive_line
              ports: SGCHANNELS/4/OUTPUT
            - iq_signal: q4/drive_line_ef
              ports: SGCHANNELS/4/OUTPUT
            - iq_signal: q5/drive_line
              ports: SGCHANNELS/5/OUTPUT
            - iq_signal: q5/drive_line_ef
              ports: SGCHANNELS/5/OUTPUT
            - iq_signal: q6/drive_line
              ports: SGCHANNELS/6/OUTPUT
            - iq_signal: q6/drive_line_ef
              ports: SGCHANNELS/6/OUTPUT
            - iq_signal: q7/drive_line
              ports: SGCHANNELS/7/OUTPUT
            - iq_signal: q7/drive_line_ef
              ports: SGCHANNELS/7/OUTPUT
        
        # device_pqsc:
        #     # - internal_clock_signal
        #     - to: device_shfqa
        #       port: ZSYNCS/3
        #     - to: device_shfsg
        #       port: ZSYNCS/1
              
    """
    device_setup = DeviceSetup.from_descriptor(
        yaml_text=descriptor,
        server_host="localhost",
        server_port=8004,
        setup_name="jw_setup",
    )
    
    return device_setup

In [3]:
device_setup = make_device_setup()

c:\Users\user\anaconda3\envs\ydkim\lib\site-packages\laboneq\dsl\device\_device_setup_generator.py:1005: FutureWarning: 'instrument_list' section is deprecated in setup descriptor, use 'instruments' instead.
  warnings.warn(


In [4]:
device_setup.logical_signal_groups

{'q0': LogicalSignalGroup(uid='q0', logical_signals={'measure_line': LogicalSignal(uid='q0/measure_line', direction=IODirection.OUT, name='measure_line', calibration=None, path='/logical_signal_groups/q0/measure_line', physical_channel=PhysicalChannel(uid=device_shfqa/qachannels_0_output, type=PhysicalChannelType.IQ_CHANNEL, name=qachannels_0_output, path=/physical_channel_groups/device_shfqa/qachannels_0_output, calibration=None)), 'acquire_line': LogicalSignal(uid='q0/acquire_line', direction=IODirection.IN, name='acquire_line', calibration=None, path='/logical_signal_groups/q0/acquire_line', physical_channel=PhysicalChannel(uid=device_shfqa/qachannels_0_input, type=PhysicalChannelType.IQ_CHANNEL, name=qachannels_0_input, path=/physical_channel_groups/device_shfqa/qachannels_0_input, calibration=None)), 'drive_line': LogicalSignal(uid='q0/drive_line', direction=IODirection.OUT, name='drive_line', calibration=None, path='/logical_signal_groups/q0/drive_line', physical_channel=Physical

In [5]:
def define_calibration(config):
    
     # the calibration object will later be applied to the device setup
    my_calibration = Calibration()


    my_calibration["/logical_signal_groups/"+ name +"/drive_line"] = \
        SignalCalibration(
           # each logical signal can have an oscillator associated with it
            oscillator=Oscillator(
                frequency=config[name]['freq'],
                modulation_type=ModulationType.HARDWARE
            ),
            local_oscillator=Oscillator(
                frequency=config["lo"]['qb_lo'],
            ),
            range=config[name]["range"]-20
        )
    
    # readout drive line
    my_calibration["/logical_signal_groups/"+name+"/measure_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=config[name]['ro'],
                modulation_type=ModulationType.SOFTWARE
            ),
            port_delay=config["rr"]['ro_delay'],
            local_oscillator=Oscillator(
                frequency=config["lo"]['ro_lo'],
            ),
           range=config["rr"]["range"]
       )
    # acquisition line
    my_calibration["/logical_signal_groups/"+name+"/acquire_line"] = \
         SignalCalibration(
            oscillator=Oscillator(
                frequency=config[name]["ro"],
                modulation_type=ModulationType.SOFTWARE
            ),
            # add an offset between the readout pulse and the start of the data acquisition - to compensate for round-trip time of readout pulse 
            port_delay=config["rr"]['ro_delay'] + config["rr"]['ro_int_delay'],
            local_oscillator=Oscillator(
                frequency=config["lo"]['ro_lo'],
            ),
            range=config["rr"]["range"]
        )
  
    return my_calibration

In [6]:
my_calibration = define_calibration(config=config)
device_setup.set_calibration(my_calibration)

In [7]:
emulate=False
my_session = Session(device_setup=device_setup)
my_session.connect(do_emulation=emulate)

2023.03.20 08:07:59.305 laboneq.controller.laboneq_logging INFO   Logging initialized from [Default inline config in laboneq.controller.laboneq_logging] logdir is d:\4. JWKim\laboneq\examples\laboneq_output\log
2023.03.20 08:07:59.306 laboneq.controller.controller  INFO   VERSION: laboneq 2.1.0
2023.03.20 08:07:59.307 laboneq.controller.devices.device_collection INFO   Connecting to data server at localhost:8004
2023.03.20 08:07:59.532 laboneq.controller.communication INFO   Connected to Zurich Instrument's Data Server version 22.08 at localhost:8004
2023.03.20 08:08:00.523 laboneq.controller.devices.device_collection INFO   Configuring clock sources
2023.03.20 08:08:00.625 laboneq.controller.devices.device_collection INFO   Clock sources configured


In [10]:
my_session.devices